#### Mapping a Subset of Table Columns

Sometimes, a `Table` object was _made available_ using the __reflection process__ described at `Reflecting Database Objects` to _load the table's structure from the database_. For such a table that has lots of columns that don't need to be referenced in the application, the `include_properties` or `exclude_properties` arguments _can specify_ that __only a subset of columns should be mapped__.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String, Table
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
user_table = Table(
    "user_table",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("user_name", String),
    Column("first_name", String),
    Column("last_name", String),
)

In [4]:
address_table = Table(
    "address_table",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("user_table.id")),
    Column("street", String),
    Column("city", String),
    Column("state", String),
    Column("zip", String),
)

In [5]:
class User(Base):
    __table__ = user_table
    __mapper_args__ = {"include_properties": ["id", "user_name"]}

...will map the `User` class to the `user_table` table, only including the `id` and `user_name` columns - the rest are not referenced. Similarly:

In [6]:
class Address(Base):
    __table__ = address_table
    __mapper_args__ = {"exclude_properties": ["street", "city", "state", "zip"]}

...will map the `Address` class to the `address_table` table, including all columns present except street, city, state, and zip.

When this mapping is used, the _columns that are not included_ __will not be referenced__ in any `SELECT` statements _emitted by_ `Query`, __nor__ will there be _any mapped attribute_ on the mapped class which represents the column; _assigning an attribute of that name_ will __have no effect__ beyond that of a normal Python attribute assignment.

In some cases, _multiple columns may have the same name_, such as when __mapping to a join of two or more tables__ that share some column name. `include_properties` and `exclude_properties` can also accommodate `Column` objects to __more accurately describe which columns should be included or excluded__.

In [7]:
class UserAddress(Base):
    __table__ = user_table.join(address_table)
    __mapper_args__ = {
        "exclude_properties": [address_table.c.id],
        "primary_key": [user_table.c.id]
    }

> ##### Note
> 
> `insert` and `update` defaults configured on individual `Column` objects, i.e. those described at Column `INSERT/UPDATE Defaults` including those configured by the `Column.default`, `Column.onupdate`, `Column.server_default` and `Column.server_onupdate` parameters, will __continue to function normally__ even if those `Column` objects are _not mapped_. This is because in the case of `Column.default` and `Column.onupdate`, the `Column` object is __still present on the underlying `Table`__, thus _allowing the default functions_ to take place when the __ORM emits__ an `INSERT` or `UPDATE`, and in the case of `Column.server_default` and `Column.server_onupdate`, the __relational database itself emits__ these defaults as a `server side behavior`.